# Wokflow con Full Bayesiana 

Para la familia de experimentos BAYESIANA ROBUSTA
del experimento colaborativo de Silvana Contreras y Sofía Scaiano.

Versión con modificaciones, basada en la notebook creada por Gustavo Denicolay.
Noviembre 2025.


## Inicializacion

In [ ]:
# limpio la memoria
Sys.time()
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

In [ ]:
PARAM <- list()
PARAM$experimento <- "wfbo_experimento_generico"
PARAM$semilla_primigenia <- 999199

In [ ]:
setwd("/content/buckets/b1/exp")
experimento_folder <- PARAM$experimento
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

## Preprocesamiento

### Generacion de la clase_ternaria

In [ ]:
Sys.time()
require( "data.table" )

# leo el dataset
dataset <- fread("~/buckets/b1/datasets/competencia_02_crudo.csv.gz" )

# calculo el periodo0 consecutivo
dsimple <- dataset[, list(
  "pos" = .I,
  numero_de_cliente,
  periodo0 = as.integer(foto_mes/100)*12 +  foto_mes%%100 )
]


# ordeno
setorder( dsimple, numero_de_cliente, periodo0 )

# calculo topes
periodo_ultimo <- dsimple[, max(periodo0) ]
periodo_anteultimo <- periodo_ultimo - 1


# calculo los leads de orden 1 y 2
dsimple[, c("periodo1", "periodo2") :=
  shift(periodo0, n=1:2, fill=NA, type="lead"),  numero_de_cliente
]

# assign most common class values = "CONTINUA"
dsimple[ periodo0 < periodo_anteultimo, clase_ternaria := "CONTINUA" ]

# calculo BAJA+1
dsimple[ periodo0 < periodo_ultimo &
  ( is.na(periodo1) | periodo0 + 1 < periodo1 ),
  clase_ternaria := "BAJA+1"
]

# calculo BAJA+2
dsimple[ periodo0 < periodo_anteultimo & (periodo0+1 == periodo1 )
  & ( is.na(periodo2) | periodo0 + 2 < periodo2 ),
  clase_ternaria := "BAJA+2"
]

# pego el resultado en el dataset original y grabo
setorder( dsimple, pos )
dataset[, clase_ternaria := dsimple$clase_ternaria ]

rm(dsimple)
gc()
Sys.time()

In [ ]:
setorder( dataset, foto_mes, clase_ternaria, numero_de_cliente)
dataset[, .N, list(foto_mes, clase_ternaria)]

### Feature Engineering Intra-Mes

Crear variables nuevas a partir de las existentes dentro del mismo registro, **sin** ir a buscar información histórica.
<br> El siguiente código es un mínimo ejemplo, agregar nuevos features a gusto

In [ ]:
# el mes 1,2, ..12 , podria servir para detectar estacionalidad
dataset[, kmes := foto_mes %% 100]

# creo un ctr_quarter que tenga en cuenta cuando
# los clientes hace 3 menos meses que estan
# ya que seria injusto considerar las transacciones medidas en menor tiempo
dataset[, ctrx_quarter_normalizado := as.numeric(ctrx_quarter) ]
dataset[cliente_antiguedad == 1, ctrx_quarter_normalizado := ctrx_quarter * 5.0]
dataset[cliente_antiguedad == 2, ctrx_quarter_normalizado := ctrx_quarter * 2.0]
dataset[cliente_antiguedad == 3, ctrx_quarter_normalizado := ctrx_quarter * 1.2]

# variable extraida de una tesis de maestria de Irlanda, se perdió el link
dataset[, mpayroll_sobre_edad := mpayroll / cliente_edad]

Sys.time()

### Feature Engineering Historico

In [ ]:
if( !require("Rcpp")) install.packages("Rcpp", repos = "http://cran.us.r-project.org")
require("Rcpp")

In [ ]:
# se calculan para los 6 meses previos el minimo, maximo y
#  tendencia calculada con cuadrados minimos
# la formula de calculo de la tendencia puede verse en
#  https://stats.libretexts.org/Bookshelves/Introductory_Statistics/Book%3A_Introductory_Statistics_(Shafer_and_Zhang)/10%3A_Correlation_and_Regression/10.04%3A_The_Least_Squares_Regression_Line
# para la maxíma velocidad esta funcion esta escrita en lenguaje C,
# y no en la porqueria de R o Python

cppFunction("NumericVector fhistC(NumericVector pcolumna, IntegerVector pdesde )
{
  /* Aqui se cargan los valores para la regresion */
  double  x[100] ;
  double  y[100] ;

  int n = pcolumna.size();
  NumericVector out( 5*n );

  for(int i = 0; i < n; i++)
  {
    //lag
    if( pdesde[i]-1 < i )  out[ i + 4*n ]  =  pcolumna[i-1] ;
    else                   out[ i + 4*n ]  =  NA_REAL ;


    int  libre    = 0 ;
    int  xvalor   = 1 ;

    for( int j= pdesde[i]-1;  j<=i; j++ )
    {
       double a = pcolumna[j] ;

       if( !R_IsNA( a ) )
       {
          y[ libre ]= a ;
          x[ libre ]= xvalor ;
          libre++ ;
       }

       xvalor++ ;
    }

    /* Si hay al menos dos valores */
    if( libre > 1 )
    {
      double  xsum  = x[0] ;
      double  ysum  = y[0] ;
      double  xysum = xsum * ysum ;
      double  xxsum = xsum * xsum ;
      double  vmin  = y[0] ;
      double  vmax  = y[0] ;

      for( int h=1; h<libre; h++)
      {
        xsum  += x[h] ;
        ysum  += y[h] ;
        xysum += x[h]*y[h] ;
        xxsum += x[h]*x[h] ;

        if( y[h] < vmin )  vmin = y[h] ;
        if( y[h] > vmax )  vmax = y[h] ;
      }

      out[ i ]  =  (libre*xysum - xsum*ysum)/(libre*xxsum -xsum*xsum) ;
      out[ i + n ]    =  vmin ;
      out[ i + 2*n ]  =  vmax ;
      out[ i + 3*n ]  =  ysum / libre ;
    }
    else
    {
      out[ i       ]  =  NA_REAL ;
      out[ i + n   ]  =  NA_REAL ;
      out[ i + 2*n ]  =  NA_REAL ;
      out[ i + 3*n ]  =  NA_REAL ;
    }
  }

  return  out;
}")

In [ ]:
# calcula la tendencia de las variables cols de los ultimos 6 meses
# la tendencia es la pendiente de la recta que ajusta por cuadrados minimos
# La funcionalidad de ratioavg es autoria de  Daiana Sparta,  UAustral  2021

TendenciaYmuchomas <- function(
    dataset, cols, ventana = 6, tendencia = TRUE,
    minimo = TRUE, maximo = TRUE, promedio = TRUE,
    ratioavg = FALSE, ratiomax = FALSE) {
  gc(verbose= FALSE)
  # Esta es la cantidad de meses que utilizo para la historia
  ventana_regresion <- ventana

  last <- nrow(dataset)

  # creo el vector_desde que indica cada ventana
  # de esta forma se acelera el procesamiento ya que lo hago una sola vez
  vector_ids <- dataset[ , numero_de_cliente ]

  vector_desde <- seq(
    -ventana_regresion + 2,
    nrow(dataset) - ventana_regresion + 1
  )

  vector_desde[1:ventana_regresion] <- 1

  for (i in 2:last) {
    if (vector_ids[i - 1] != vector_ids[i]) {
      vector_desde[i] <- i
    }
  }
  for (i in 2:last) {
    if (vector_desde[i] < vector_desde[i - 1]) {
      vector_desde[i] <- vector_desde[i - 1]
    }
  }

  for (campo in cols) {
    nueva_col <- fhistC(dataset[, get(campo)], vector_desde)

    if (tendencia) {
      dataset[, paste0(campo, "_tend", ventana) :=
        nueva_col[(0 * last + 1):(1 * last)]]
    }

    if (minimo) {
      dataset[, paste0(campo, "_min", ventana) :=
        nueva_col[(1 * last + 1):(2 * last)]]
    }

    if (maximo) {
      dataset[, paste0(campo, "_max", ventana) :=
        nueva_col[(2 * last + 1):(3 * last)]]
    }

    if (promedio) {
      dataset[, paste0(campo, "_avg", ventana) :=
        nueva_col[(3 * last + 1):(4 * last)]]
    }

    if (ratioavg) {
      dataset[, paste0(campo, "_ratioavg", ventana) :=
        get(campo) / nueva_col[(3 * last + 1):(4 * last)]]
    }

    if (ratiomax) {
      dataset[, paste0(campo, "_ratiomax", ventana) :=
        get(campo) / nueva_col[(2 * last + 1):(3 * last)]]
    }
  }
}

In [ ]:
# Feature Engineering Historico
# Creacion de LAGs
setorder(dataset, numero_de_cliente, foto_mes)

# todo es lagueable, menos la primary key y la clase
cols_lagueables <- copy( setdiff(
  colnames(dataset),
  c("numero_de_cliente", "foto_mes", "clase_ternaria")
))

# https://rdrr.io/cran/data.table/man/shift.html

# lags de orden 1
dataset[,
  paste0(cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"),
  by= numero_de_cliente,
  .SDcols= cols_lagueables
]

# lags de orden 2
dataset[,
  paste0(cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag"),
  by= numero_de_cliente,
  .SDcols= cols_lagueables
]

# agrego los delta lags
for (vcol in cols_lagueables)
{
  dataset[, paste0(vcol, "_delta1") := get(vcol) - get(paste0(vcol, "_lag1"))]
  dataset[, paste0(vcol, "_delta2") := get(vcol) - get(paste0(vcol, "_lag2"))]
}

Sys.time()

In [ ]:
# parametros de Feature Engineering Historico de Tendencias
PARAM$FE_hist$Tendencias$run <- TRUE
PARAM$FE_hist$Tendencias$ventana <- 6
PARAM$FE_hist$Tendencias$tendencia <- TRUE
PARAM$FE_hist$Tendencias$minimo <- FALSE
PARAM$FE_hist$Tendencias$maximo <- FALSE
PARAM$FE_hist$Tendencias$promedio <- FALSE
PARAM$FE_hist$Tendencias$ratioavg <- FALSE
PARAM$FE_hist$Tendencias$ratiomax <- FALSE

In [ ]:
# aqui se agregan las tendencias de los ultimos 6 meses

cols_lagueables <- intersect(cols_lagueables, colnames(dataset))
setorder(dataset, numero_de_cliente, foto_mes)

if( PARAM$FE_hist$Tendencias$run) {
    TendenciaYmuchomas(dataset,
    cols = cols_lagueables,
    ventana = PARAM$FE_hist$Tendencias$ventana, # 6 meses de historia
    tendencia = PARAM$FE_hist$Tendencias$tendencia,
    minimo = PARAM$FE_hist$Tendencias$minimo,
    maximo = PARAM$FE_hist$Tendencias$maximo,
    promedio = PARAM$FE_hist$Tendencias$promedio,
    ratioavg = PARAM$FE_hist$Tendencias$ratioavg,
    ratiomax = PARAM$FE_hist$Tendencias$ratiomax
  )
}

ncol(dataset)
Sys.time()

In [ ]:
ncol(dataset)
colnames(dataset)

## Modelado

### Target Engineering

In [ ]:
# paso la clase a binaria que tome valores {0,1}  enteros
#  BAJA+1 y BAJA+2  son  1,   CONTINUA es 0
#  a partir de ahora ya NO puedo cortar  por prob(BAJA+2) > 1/40

dataset[, clase01 := ifelse(clase_ternaria %in% c("BAJA+2","BAJA+1"), 1L, 0L)]

### Training Strategy

In [ ]:
PARAM$trainingstrategy$testing <- c(202104)

PARAM$trainingstrategy$training <- c(
  201901, 201902, 201903, 201904, 201905, 201906,
  201907, 201908, 201909, 201910, 201911, 201912,
  202001, 202002, 202003, 202004, 202005, 202006,
  202007, 202008, 202009, 202010, 202011, 202012,
  202101, 202102
  #, 202103, 202104
)

PARAM$trainingstrategy$undersampling <- 0.05


PARAM$trainingstrategy$positivos <- c( "BAJA+1", "BAJA+2")

In [ ]:
# los campos en los que se entrena
campos_buenos <- copy( setdiff(
    colnames(dataset), c("clase_ternaria","clase01","azar"))
)

#### Registros  cambio las proporciones de POS/NEG

In [ ]:
# Undersampling, van todos los "BAJA+1" y "BAJA+2" y solo algunos "CONTINIA"

set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset[, azar := runif(nrow(dataset))]
dataset[, training := 0L]

dataset[  foto_mes %in%  PARAM$trainingstrategy$training &
  (azar <= PARAM$trainingstrategy$undersampling | clase_ternaria %in% c("BAJA+1", "BAJA+2")),
  training := 1L
]


### Optimizacion de Hipeparámetros

Se optimizan los hiperparámetros maximizando la ganancia.

In [ ]:
if( !require("lightgbm")) install.packages("lightgbm")
require("lightgbm")

In [ ]:
dtrain <- lgb.Dataset(
  data= data.matrix(dataset[training == 1L, campos_buenos, with = FALSE]),
  label= dataset[training == 1L, clase01],
  free_raw_data= TRUE
)

cat("filas", nrow(dtrain), "columnas", ncol(dtrain), "\n")
Sys.time()

In [ ]:
# defino los datos de testing
dataset_test <- dataset[foto_mes %in% PARAM$trainingstrategy$testing]

# precalculo el campo de la ganancia
dataset_test[, gan := -20000.0 ]
dataset_test[ clase_ternaria=="BAJA+2", gan := 780000]

# precalculo la test_matrix
test_matrix <- data.matrix(dataset_test[, campos_buenos, with= FALSE])

cat("filas", nrow(dataset_test), "columnas", ncol(dataset_test), "\n")
Sys.time()

In [ ]:
# paquetes necesarios para la Bayesian Optimization
if(!require("DiceKriging")) install.packages("DiceKriging")
require("DiceKriging")

if(!require("mlrMBO")) install.packages("mlrMBO")
require("mlrMBO")

In [ ]:
# Especificacion de la cantidad de iteraciones de la Bayesian Optimization
# 50 es razonable
PARAM$hipeparametertuning$BO_iteraciones <- 20 # un 50 seria mas razonable

In [ ]:
# parametros fijos del LightGBM
PARAM$lgbm$param_fijos <- list(
  objective= "binary",
  metric= "auc",
  first_metric_only= TRUE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  verbosity= -100,
  force_row_wise= TRUE, # para evitar warning
  seed= PARAM$semilla_primigenia,
  extra_trees = FALSE,

  max_depth = -1L, # -1 significa no limitar,  por ahora lo dejo fijo
  min_gain_to_split = 0.0, # min_gain_to_split >= 0.0
  min_sum_hessian_in_leaf = 0.001, #  min_sum_hessian_in_leaf >= 0.0
  lambda_l1 = 0.0, # lambda_l1 >= 0.0
  lambda_l2 = 0.0, # lambda_l2 >= 0.0

  bagging_fraction = 1.0, # 0.0 < bagging_fraction <= 1.0
  pos_bagging_fraction = 1.0, # 0.0 < pos_bagging_fraction <= 1.0
  neg_bagging_fraction = 1.0, # 0.0 < neg_bagging_fraction <= 1.0
  is_unbalance = FALSE, #
  scale_pos_weight = 1.0, # scale_pos_weight > 0.0

  drop_rate = 0.1, # 0.0 < neg_bagging_fraction <= 1.0
  max_drop = 50, # <=0 means no limit
  skip_drop = 0.5, # 0.0 <= skip_drop <= 1.0

  max_bin= 31
)

In [ ]:
# Notar que se recorren algunos hiperparametros en forma logaritmica
#   y que con forbidden se tiene en cuenta el juego que hay entre min_data_in_leaf y num_leaves

PARAM$hipeparametertuning$hs <- makeParamSet(
  makeNumericParam("num_iterations", lower= 0.0, upper= 11.1, trafo= function(x) as.integer( round(2^x)) ),
  makeNumericParam("learning_rate", lower= -8.0, upper= -1.0, trafo= function(x) 2^x ),
  makeNumericParam("feature_fraction", lower= 0.05, upper= 1.0 ),
  makeNumericParam("min_data_in_leaf", lower= 0.0, upper= log2(nrow(dtrain)/2), trafo= function(x) as.integer(round(2^x)) ),
  makeNumericParam("num_leaves", lower= 1.0, upper= 10.0, trafo= function(x) as.integer(round(2^x)) ),
  forbidden= quote( (2^min_data_in_leaf)*(2^num_leaves) > nrow(dtrain) )
)

In [ ]:
PARAM$hipeparametertuning$ksemillerio <- 20L
PARAM$hipeparametertuning$repe <- 3L


In [ ]:
if(!require("primes")) install.packages("primes")
require("primes")

In [ ]:
primos <- generate_primes(min = 100000, max = 1000000)
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
# me quedo con PARAM$semillerio  primos al azar
PARAM$BO$semillas <- sample(primos)[seq( PARAM$hipeparametertuning$ksemillerio * PARAM$hipeparametertuning$repe )]

cat( PARAM$BO$semillas)

In [ ]:
if(!require("rlist")) install.packages("rlist")
require("rlist")

In [ ]:
# logueo al archivo BO_log.txt
loguear  <- function( reg, arch=NA, verbose=TRUE )
{
  t0 <- Sys.time()
  archivo <- arch
  if( is.na(arch) ) archivo <- paste0( folder, substitute( reg), ext )


  if( !file.exists( archivo ) )
  {
    # Escribo los titulos
    linea  <- paste0( "fecha\t", 
                      paste( list.names(reg), collapse="\t" ), "\n" )

    cat( linea, file=archivo )
  }

  # escribo el registro
  linea  <- paste0( format(t0, "%Y%m%d.%H%M%S"),  "\t",     # la fecha y hora
                    gsub( ", ", "\t", toString( reg ) ),  "\n" )

  cat( linea, file=archivo, append=TRUE )  # grabo al archivo

  if( verbose )  cat( linea )   # imprimo por pantalla
}

In [ ]:
# esto esta en una funcion para que el garbage collector lo libere
# entrena, aplica el modelo a testing, y devuelve el vector de probabilidades

OneTrainPredict <- function(param_completo) {
    
  modelo <- lgb.train(
    data= dtrain,
    param= param_completo
  )
  gmodelo <<- modelo

  # aplico el modelo a los datos nuevos
  pred <- predict(
    modelo,
    test_matrix
  )

  return( pred )
}

In [ ]:
# En el argumento x llegan los parmaetros de la bayesiana
#  devuelve la ganancia en datos de testing

# aqui se ira guardando la mejor iteracion de la bayesiana
gmejor <- list()
gmejor$iter <- 0
gmejor$gan <- -Inf

giter <- 0
if( file.exists("BO_log.txt") ){
  tb_BO <- fread("BO_log.txt")
  giter <- nrow(tb_BO) -1 
}

EstimarGanancia_lightgbm <- function(x) {

  giter <<- giter + 1
  # x pisa (o agrega) a param_fijos
  param_completo <- modifyList(PARAM$lgbm$param_fijos, x)

  vgan_mesetas <- c()  # las ganancias, tengo repe de ellas

  # loop de las repeticionies
  for( repe in seq( PARAM$hipeparametertuning$repe ) )
  {
     desde <- (PARAM$hipeparametertuning$repe-1)*PARAM$hipeparametertuning$ksemillerio + 1
     hasta <- desde + PARAM$hipeparametertuning$ksemillerio -1
     rsemillas <- PARAM$BO$semillas[ desde:hasta ]

     # vector inicial de probabilidades
     vpred_acum <- rep( 0.0, nrow(dataset_test) )

     # loop del semillerio
     for( sem in rsemillas ) # itero semillerio
     {
        param_completo$seed <- sem  # asigno se semilla
        vpred_acum <- vpred_acum + OneTrainPredict( param_completo )
        
        gc(full= TRUE, verbose= FALSE)
     }

     # Calculo de ganancia suavizada de la meseta
     tb_prediccion <- dataset_test[, list(gan)]
     tb_prediccion[, prob := vpred_acum ]

     setorder(tb_prediccion, -prob)
     tb_prediccion[, gan_acum := cumsum(gan)]

     # la meseta es un punto, mil para la izquierda, otros mil para la derecha
     tb_prediccion[, gan_meseta :=
       frollmean(
         x= gan_acum, n= 2001, align= "center",
         na.rm= TRUE, hasNA= TRUE
      )
     ]

     vgan_mesetas <- c(vgan_mesetas, tb_prediccion[, max(gan_meseta, na.rm = TRUE)] )
  }

  gan_mesetas_prom <- mean( vgan_mesetas ) 

  if( gan_mesetas_prom > gmejor$gan ){
    gmejor$gan <<- gan_mesetas_prom
    gmejor$iter <<- giter

    # hrabo importancia de variables
    fwrite( lgb.importance(gmodelo),
      file= paste0("impo_", giter, ".txt"),
      sep= "\t"
    )
  }

  # datos qeu voy a loguear
  xx <- copy(param_completo)
  xx$iter <- giter
  xx$metrica_mejor <- gmejor$gan
  xx$metrica_sd <- sd(vgan_mesetas)
  xx$metrica <- gan_mesetas_prom

  loguear( xx, "BO_log.txt")
  set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")  # le reordeno a mlrMBO

  return( gan_mesetas_prom ) #tiempo_corrida) )
}

In [ ]:
# Aqui comienza la configuracion de la Bayesian Optimization
#  es compleja la configuracion de una Bayesian Optimization

# en este archivo quedan la evolucion binaria de la BO
kbayesiana <- "bayesiana.RDATA"

funcion_optimizar <- EstimarGanancia_lightgbm # la funcion que voy a maximizar

configureMlr(show.learner.output= FALSE)

# configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
# por favor, no desesperarse por lo complejo

obj.fun <- makeSingleObjectiveFunction(
  fn= funcion_optimizar, # la funcion que voy a maximizar
  minimize= FALSE, # estoy Maximizando la ganancia
  noisy= TRUE,
  par.set= PARAM$hipeparametertuning$hs, # definido al comienzo del programa
  has.simple.signature= FALSE # paso los parametros en una lista
)

# cada 600 segundos guardo el resultado intermedio
ctrl <- makeMBOControl(
  save.on.disk.at.time= 600, # se graba cada 600 segundos
  save.file.path= kbayesiana
) # se graba cada 600 segundos

# indico la cantidad de iteraciones que va a tener la Bayesian Optimization
ctrl <- setMBOControlTermination(
  ctrl,
  iters= PARAM$hipeparametertuning$BO_iteraciones
) # cantidad de iteraciones

# defino el método estandar para la creacion de los puntos iniciales,
# los "No Inteligentes"
ctrl <- setMBOControlInfill(ctrl, crit= makeMBOInfillCritEI())

# establezco la funcion que busca el maximo
surr.km <- makeLearner(
  "regr.km",
  predict.type= "se",
  covtype= "matern3_2",
  control= list(trace= TRUE)
)

Sys.time()

#### Corrida de la Bayesian Optimization

In [ ]:
# inicio la optimizacion bayesiana, retomando si ya existe
# es la celda mas lenta de todo el notebook

if (!file.exists(kbayesiana)) {
  bayesiana_salida <- mbo(obj.fun, learner= surr.km, control= ctrl)
} else {
  bayesiana_salida <- mboContinue(kbayesiana) # retomo en caso que ya exista
}

Sys.time()

In [ ]:
Sys.time()

## Produccion

### Final Training Strategy

In [ ]:
PARAM$train_final$future <- c(202108)

PARAM$train_final$training <- c(
  201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908, 201909, 201910, 201911, 201912
, 202001, 202002, 202003
, 202004, 202005, 202006, 202007, 202008, 202009, 202010
, 202011, 202012
, 202101, 202102, 202103, 202104
#, 202105, 202106
)

PARAM$train_final$undersampling <- 0.01 #modifiqué decía 0.1

In [ ]:
# se filtran los meses donde se entrena el modelo final
dataset_train_final <- dataset[foto_mes %in% PARAM$train_final$training]

#### Registros cambio las proporciones de POS/NEG

In [ ]:
# Undersampling, van todos los "BAJA+1" y "BAJA+2" y solo algunos "CONTINIA"

set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset_train_final[, azar := runif(nrow(dataset_train_final))]
dataset_train_final[, training := 0L]

dataset_train_final[
  (azar <= PARAM$train_final$undersampling | clase_ternaria %in% c("BAJA+1", "BAJA+2")),
  training := 1L
]

dataset_train_final[, azar:= NULL] # elimino la columna azar

### Target Engineering

In [ ]:
# paso la clase a binaria que tome valores {0,1}  enteros
#  BAJA+1 y BAJA+2  son  1,   CONTINUA es 0
#  a partir de ahora ya NO puedo cortar  por prob(BAJA+2) > 1/40

dataset_train_final[,
  clase01 := ifelse(clase_ternaria %in% c("BAJA+2","BAJA+1"), 1L, 0L)
]

### Adaptacion Hiperparametros Optimos

Solamente escalo min_data_in_leaf  por  nrow(dataset_train_final) / nrow(dtrain)

In [ ]:
# leo el archivo donde quedaron los hiperparametros optimos
tb_BO <-  fread("BO_log.txt")
setorder( tb_BO, -metrica)  # ordeno por metrica descendente
tb_BO[1]

In [ ]:
# en la tabla ademas de los parametros del LightGBM, hay campos de salida
param_lgbm <- union( names(PARAM$lgbm$param_fijos),  names(PARAM$hipeparametertuning$hs$pars) )

PARAM$train_final$param_mejores <- as.list( tb_BO[1, param_lgbm, with=FALSE])

PARAM$train_final$param_mejores$min_data_in_leaf <- as.integer( round(PARAM$train_final$param_mejores$min_data_in_leaf * nrow(dataset_train_final[training == 1L]) / nrow(dtrain)))

cat( tb_BO[1, min_data_in_leaf] , PARAM$train_final$param_mejores$min_data_in_leaf, "\n")
PARAM$train_final$param_mejores

### Final Models

Aqui SIEMPRE voy a hacer un semillerio, independientemente de si en la Bayesian Optimization calculé un semillerio en cada iteración.
<br> Entreno un LightGBM para cada semilla,  y guardo el modelo dentro de la carpeta  **modelitos**
<br> Intencionalmente en una primera etapá se generan los modelos y graban, y en una segunda etapa se leen eso modelos y se aplican a los datos del futuro

In [ ]:
# Semillerio Final
PARAM$train_final$ksemillerio  <- 30 #MODIFICAR SEGUN EXPERIMENTO


In [ ]:
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
PARAM$train_final$semillas <- sample(primos)[seq( PARAM$train_final$ksemillerio )]
PARAM$train_final$semillas

In [ ]:
# dejo los datos en formato LightGBM
dtrain_final <- lgb.Dataset(
  data= data.matrix(dataset_train_final[training == 1L, campos_buenos, with= FALSE]),
  label= dataset_train_final[training == 1L, clase01],
  free_raw_data= FALSE
)

cat("filas", nrow(dtrain_final), "columnas", ncol(dtrain_final), "\n")
Sys.time()

In [ ]:
# genero los modelitos

dir.create( "modelitos", showWarnings= FALSE)

param_completo <- copy( PARAM$train_final$param_mejores)

for( sem in PARAM$train_final$semillas ) {
  arch_modelo <- sprintf("./modelitos/mod_%s_%s.txt", PARAM$experimento, sem)
  if( !file.exists( arch_modelo ) )
  {
    param_completo$seed <- sem

    modelito <- lgb.train(
      data= dtrain_final,
      param= param_completo
    )

    lgb.save( modelito, filename= arch_modelo)
    rm(modelito)
     gc()
    cat("✓ Modelo guardado seed", sem, "\n")
  } else {
    cat("→ Ya existía, salto seed", sem, "\n")
  }
}
cat("\nListo. Si no había pendientes, no se recalculó nada.\n")

Sys.time()




### Scoring

Se hace el predict() del modelo en los datos del futuro

In [ ]:
# aplico el modelo a los datos sin clase
dfuture <- dataset[foto_mes %in% PARAM$train_final$future ]
mfuture <- data.matrix(dfuture[, campos_buenos, with= FALSE])

stopifnot(nrow(dfuture) > 0, ncol(mfuture) > 0, length(PARAM$train_final$semillas) > 0)

vpred_acum <- rep(0.0, nrow(dfuture))
qacumulados <- 0L

for(sem in PARAM$train_final$semillas) {
  arch_modelo <- sprintf("./modelitos/mod_%s_%s.txt", PARAM$experimento, sem)
  modelo_final <- lgb.load(arch_modelo)
  vpred_individual <- predict(modelo_final, mfuture)

  tb_pred_seed <- dfuture[, .(numero_de_cliente, foto_mes)]
  tb_pred_seed[, prob := vpred_individual ]
  fwrite(tb_pred_seed,
         file = sprintf("prediccion_future_%s_%s.txt", PARAM$experimento, sem),
         sep  = "\t")

  vpred_acum <- vpred_acum + vpred_individual
  qacumulados <- qacumulados + 1L
}

vpred_acum <- vpred_acum / qacumulados

tb_prediccion <- dfuture[, .(numero_de_cliente, foto_mes)]
tb_prediccion[, prob := vpred_acum ]
fwrite(tb_prediccion,
       file = sprintf("prediccion_future_%s.txt", PARAM$experimento),
       sep  = "\t")



### Clasificacion

Se tomó la decisión de enviar a los 11000 registros con mayor probabilidad de POS={"BAJA+1","BAJA+"}
<br> esto se determinó en forma artesanal analizando meses anterior
<br> esta es una muy importante decisión 

In [ ]:
# genero archivos con los  "envios" mejores
dir.create("kaggle", showWarnings=FALSE)

# ordeno por probabilidad descendente
setorder(tb_prediccion, -prob)

envios <- 11000
tb_prediccion[, Predicted := 0L] # seteo inicial a 0
tb_prediccion[1:envios, Predicted := 1L] # marco los primeros

archivo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_", envios, ".csv")

# grabo el archivo
fwrite(tb_prediccion[, list(numero_de_cliente, Predicted)],
  file= archivo_kaggle,
  sep= ","
)

In [ ]:
cat("\n=== Guardando parámetros del experimento ===\n")
if (!requireNamespace("yaml", quietly = TRUE)) install.packages("yaml")

# Construir primero la versión "plana"
PARAM_export <- list(
  experimento  = PARAM$experimento,
  train_final  = list(
    future        = PARAM$train_final$future,
    training      = PARAM$train_final$training,
    undersampling = PARAM$train_final$undersampling,
    semillas      = PARAM$train_final$semillas,
    param_mejores = PARAM$train_final$param_mejores
  ),
  cortes         = if ("cortes" %in% names(PARAM)) PARAM$cortes else NULL,
  qcanaritos     = if ("qcanaritos" %in% names(PARAM)) PARAM$qcanaritos else NULL,
  campos_buenos  = if (exists("campos_buenos")) campos_buenos else NULL
)

nombre_archivo_param <- sprintf("PARAM_plano_exp_%s.yml", PARAM$experimento)
yaml::write_yaml(PARAM_export, file = nombre_archivo_param)
cat("Archivo de parámetros guardado exitosamente en:", nombre_archivo_param, "\n")

# (Opcional) si querés además un “global”:
nombre_archivo_global_param <- sprintf("PARAM_global_exp_%s.yml", PARAM$experimento)
yaml::write_yaml(PARAM_export, file = nombre_archivo_global_param)
cat("Archivo global de parámetros guardado exitosamente en:", nombre_archivo_global_param, "\n")


In [ ]:
Sys.time()

# TEST

In [ ]:
# particionar agrega una columna llamada fold a un dataset
#   que consiste en una particion estratificada segun agrupa
# particionar( data=dataset, division=c(70,30),
#  agrupa=clase_ternaria, seed=semilla)   crea una particion 70, 30

PARAM$semilla_kaggle <- 314159

particionar <- function(data, division, agrupa= "", campo= "fold", start= 1, seed= NA) {
  if (!is.na(seed)) set.seed(seed, "L'Ecuyer-CMRG")

  bloque <- unlist(mapply(
    function(x, y) {rep(y, x)},division, seq(from= start, length.out= length(division))))

  data[, (campo) := sample(rep(bloque,ceiling(.N / length(bloque))))[1:.N],by= agrupa]
}

# iniciliazo el dataset de realidad, para medir ganancia
realidad_inicializar <- function( pfuture, pparam) {

  # datos para verificar la ganancia
  drealidad <- pfuture[, list(numero_de_cliente, foto_mes, clase_ternaria)]

  particionar(drealidad,
    division= c(3, 7),
    agrupa= "clase_ternaria",
    seed= PARAM$semilla_kaggle 
  )

  return( drealidad )
}

# evaluo ganancia en los datos de la realidad

realidad_evaluar <- function( prealidad, pprediccion) {

  prealidad[ pprediccion,
    on= c("numero_de_cliente", "foto_mes"),
    predicted:= i.Predicted
  ]

  tbl <- prealidad[, list("qty"=.N), list(fold, predicted, clase_ternaria)]

  res <- list()
  res$public  <- tbl[fold==1 & predicted==1L, sum(qty*ifelse(clase_ternaria=="BAJA+2", 780000, -20000))]/0.3
  res$private <- tbl[fold==2 & predicted==1L, sum(qty*ifelse(clase_ternaria=="BAJA+2", 780000, -20000))]/0.7
  res$total <- tbl[predicted==1L, sum(qty*ifelse(clase_ternaria=="BAJA+2", 780000, -20000))]

  prealidad[, predicted:=NULL]
  return( res )
}


### predicciones false future

In [ ]:
#`dataset`, `PARAM`, `campos_buenos` y `realidad_evaluar` cargados

PARAM$false_future <- c(202106) 
d_false_future <- dataset[foto_mes %in% PARAM$false_future]

m_false_future <- data.matrix(d_false_future[, campos_buenos, with = FALSE])

drealidad <- realidad_inicializar(d_false_future, PARAM) 


cat("\n=== Generando predicciones del Ensemble y por Semilla ===\n")

vpred_acum_false <- rep(0.0, nrow(d_false_future))
qacumulados_false <- 0
todas_las_predicciones_semilla <- list() 

for (semilla in PARAM$train_final$semillas) {
    arch_modelo <- sprintf("./modelitos/mod_%s_%s.txt", PARAM$experimento, semilla)
    if (file.exists(arch_modelo)) {
        modelo_individual <- lgb.load(arch_modelo) 
        vpred_individual <- predict(modelo_individual, m_false_future) 


        vpred_acum_false <- vpred_acum_false + vpred_individual
        qacumulados_false <- qacumulados_false + 1

        # Crear y guardar predicción individual
        tb_prediccion_semilla <- d_false_future[, list(numero_de_cliente, foto_mes)]
        tb_prediccion_semilla[, prob := vpred_individual]
        nombre_archivo_salida_semilla <- paste0("prediccion_false_future_seed_", PARAM$experimento, "_", semilla, ".txt")
        fwrite(tb_prediccion_semilla, file = nombre_archivo_salida_semilla, sep = "\t")
        cat(" -> Semilla", semilla, "predicción guardada en:", nombre_archivo_salida_semilla, "\n")
        
        
        todas_las_predicciones_semilla[[as.character(semilla)]] <- tb_prediccion_semilla
        
    } else {
        cat(" -> ADVERTENCIA: Modelo no encontrado para la semilla:", semilla, "\n")
    }
}

# Calcular y guardar predicción del Ensemble Promedio
if (qacumulados_false > 0) {
    vpred_acum_false <- vpred_acum_false / qacumulados_false 
    tb_prediccion_false_future_prom <- d_false_future[, list(numero_de_cliente, foto_mes)]
    tb_prediccion_false_future_prom[, prob := vpred_acum_false]

    fwrite(tb_prediccion_false_future_prom,
           file = paste0("prediccion_false_future_ensemble_", PARAM$experimento, ".txt"), 
           sep = "\t"
    )
    cat("\n Predicción del Ensemble Promedio guardada en: prediccion_false_future_ensemble_", PARAM$experimento, ".txt\n")
} else {
    stop("No se encontraron modelos para generar predicciones del ensemble.")
}





In [ ]:
# GANANCIASEN TEST  (semillas + ensemble + derivado )
cat("\n=== Calculando Ganancias de TODAS las Semillas en TODOS los Cortes ===\n")

ganancias_completas <- data.table()
PARAM$cortes <- seq(7000, 16000, by = 500)


for (semilla_name in names(todas_las_predicciones_semilla)) {

  tb_pred <- todas_las_predicciones_semilla[[semilla_name]]
  setorder(tb_pred, -prob)

  for (corte in PARAM$cortes) {
    tb_temp <- copy(tb_pred)
    tb_temp[, Predicted := 0L]
    tb_temp[1:corte, Predicted := 1L]

    res <- realidad_evaluar(drealidad, tb_temp)

    ganancias_completas <- rbind(
      ganancias_completas,
      data.table(
        tipo = paste0("Seed ", semilla_name),
        corte = corte,
        ganancia = res$total
      )
    )
  }

  cat("Semilla", semilla_name, "completada\n")
}


cat("Calculando ganancias para el Ensemble promedio\n")
setorder(tb_prediccion_false_future_prom, -prob)
tb_ensemble <- copy(tb_prediccion_false_future_prom)
mejor_ganancia_ensemble <- -Inf
mejor_corte <- NA

for (corte in PARAM$cortes) {
  tb_ensemble[, Predicted := 0L]
  tb_ensemble[1:corte, Predicted := 1L]

  res <- realidad_evaluar(drealidad, tb_ensemble)

 
  ganancias_completas <- rbind(
    ganancias_completas,
    data.table(
      tipo = "Ensemble",
      corte = corte,
      ganancia = res$total
    )
  )


  cat("Envios=", corte, "\t",
      " TOTAL=",   format(res$total,   big.mark = ","),
      " Public=",  format(res$public,  big.mark = ","),
      " Private=", format(res$private, big.mark = ","),
      "\n", sep = "")

  if (res$total > mejor_ganancia_ensemble) {
    mejor_ganancia_ensemble <- res$total
    mejor_corte <- corte
  }
}
cat("Corte óptimo del Ensemble (para línea roja):", mejor_corte, "envíos\n")


archivo_completo <- paste0("ganancias_todas_semillas_cortes_", PARAM$experimento, ".txt")
fwrite(ganancias_completas, file = archivo_completo, sep = "\t")
cat("\nGanancias completas (por corte) guardadas en:", archivo_completo, "\n")


resultados_ganancia_ensemble <- ganancias_completas[
  tipo == "Ensemble",
  .(envios = corte, ganancia_total = ganancia)
][order(envios)]

nombre_archivo_ganancias_ensemble <- paste0("ganancias_ensemble_exp_", PARAM$experimento, ".txt")
fwrite(resultados_ganancia_ensemble, file = nombre_archivo_ganancias_ensemble, sep = "\t")
cat("Ganancias del Ensemble (derivadas) guardadas en:", nombre_archivo_ganancias_ensemble, "\n")



In [ ]:
library(ggplot2)
library(scales)

semillas_plot <- ganancias_completas[tipo != "Ensemble"]
ensemble_plot <- ganancias_completas[tipo == "Ensemble"]


p_lineas <- ggplot(semillas_plot, aes(x = corte, y = ganancia, color = tipo, group = tipo)) +
    
    geom_line(size = 0.8, alpha = 0.7) +
    
    geom_line(data = ensemble_plot, 
          aes(x = corte, y = ganancia, color = "Ensemble"),
          size = 1.2, alpha = 1) +
    scale_color_discrete(breaks = c("Ensemble", unique(semillas_plot$tipo)))+


    geom_vline(xintercept = mejor_corte, linetype = "dashed", color = "red", size = 1) +
    
    # Escalas y Etiquetas
    scale_y_continuous(labels = scales::comma) +
    scale_x_continuous(labels = scales::comma) +
    labs(
        title = paste("Curvas de Ganancia por Semilla y Ensemble - EXP", PARAM$experimento),
        subtitle = paste("Línea roja: corte óptimo del Ensemble =", mejor_corte),
        x = "Corte (cantidad de envíos)",
        y = "Ganancia",
        color = "Modelo"
    ) +
    theme_minimal() +
    scale_color_discrete(breaks = c("Ensemble", unique(semillas_plot$tipo)))


print(p_lineas)


nombre_archivo_png <- paste0("curva_ganancia_semilla_corte_", PARAM$experimento, ".png")

ggsave(nombre_archivo_png, p_lineas, width = 14, height = 8, dpi = 300) 

cat("Gráfico de Ganancia por Corte guardado en:", nombre_archivo_png, "\n")

### grafico acumulado ensamble

In [ ]:
# Ordenar por probabilidad descendente
setorder(tb_prediccion_false_future_prom, -prob)

# Calcular ganancia acumulada
tb_prediccion_false_future_prom[, indice := 1:.N]

# Unir con drealidad para tener clase_ternaria
tb_ganancia <- merge(
  tb_prediccion_false_future_prom[, .(numero_de_cliente, foto_mes, prob, indice)],
  drealidad[, .(numero_de_cliente, foto_mes, clase_ternaria)],
  by = c("numero_de_cliente", "foto_mes")
)

setorder(tb_ganancia, indice)

tb_ganancia[, ganancia_individual := ifelse(clase_ternaria == "BAJA+2", 780000, -20000)]
tb_ganancia[, ganancia_acumulada := cumsum(ganancia_individual)]

ganancia_maxima <- max(tb_ganancia$ganancia_acumulada)
indice_maximo <- tb_ganancia[ganancia_acumulada == ganancia_maxima, indice][1]

umbral_ganancia <- ganancia_maxima * 0.66
tb_filtrada <- tb_ganancia[ganancia_acumulada >= umbral_ganancia]

p_acumulada <- ggplot(tb_filtrada, aes(x = indice, y = ganancia_acumulada)) +
  geom_line(color = "blue", linewidth = 1.2) +
  geom_point(data = tb_ganancia[indice == indice_maximo], 
             aes(x = indice, y = ganancia_acumulada),
             color = "red", size = 3) +
  annotate("text", 
           x = indice_maximo, 
           y = ganancia_maxima * 1.05,
           label = paste0("Ganancia Máxima\n", format(ganancia_maxima, big.mark = ",", scientific = FALSE)),
           color = "red", 
           fontface = "bold") +
  annotate("segment",
           x = indice_maximo, y = ganancia_maxima,
           xend = indice_maximo, yend = ganancia_maxima * 1.04,
           arrow = arrow(length = unit(0.3, "cm")),
           color = "red") +
  scale_x_continuous(labels = scales::comma) +
  scale_y_continuous(labels = scales::comma) +
  labs(
    title = paste0("Ganancia acumulada por orden de predicción (filtrada) - EXP ", PARAM$experimento),
    x = "Clientes ordenados por probabilidad",
    y = "Ganancia Acumulada"
  ) +
  theme_minimal() +
  theme(
    plot.title = element_text(face = "bold", size = 14),
    panel.grid.minor = element_line(linewidth = 0.3, linetype = "dotted")
  )

print(p_acumulada)

nombre_archivo_acum <- paste0("curva_ganancia_acumulada_exp_", PARAM$experimento, ".png")
ggsave(nombre_archivo_acum, p_acumulada, width = 14, height = 8, dpi = 300)

cat("Ganancia máxima:", format(ganancia_maxima, big.mark = ","), "\n")
cat("Corte ideal por cliente:", indice_maximo, "\n")

### grafico acumulado semillas

In [ ]:
calc_curva_acum_filtrada <- function(tb_pred, drealidad, umbral_ganancia) {

  tb_ganancia <- copy(tb_pred)
  setorder(tb_ganancia, -prob)
  

  tb_ganancia[, indice := 1:.N]
  
  tb_ganancia <- merge(
    tb_ganancia[, .(numero_de_cliente, foto_mes, prob, indice)],
    drealidad[, .(numero_de_cliente, foto_mes, clase_ternaria)],
    by = c("numero_de_cliente", "foto_mes")
  )
  
  setorder(tb_ganancia, indice)
  tb_ganancia[, ganancia_individual := ifelse(clase_ternaria == "BAJA+2", 780000, -20000)]
  tb_ganancia[, ganancia_acumulada := cumsum(ganancia_individual)]
  
  ganancia_maxima <- max(tb_ganancia$ganancia_acumulada)
  
  if (missing(umbral_ganancia)) {
      umbral_ganancia <- ganancia_maxima * 0.66
  }
  
  tb_filtrada <- tb_ganancia[ganancia_acumulada >= umbral_ganancia]
  
  return(list(
      data_full = tb_ganancia, 
      data_plot = tb_filtrada, 
      maxima = ganancia_maxima
  ))
}


cat("\n=== Calculando Curvas de Ganancia Acumulada ===\n")

res_ensemble <- calc_curva_acum_filtrada(tb_prediccion_false_future_prom, drealidad)
ganancia_maxima <- res_ensemble$maxima
indice_maximo <- res_ensemble$data_full[ganancia_acumulada == ganancia_maxima, indice][1]
umbral_ganancia_base <- ganancia_maxima * 0.66 

cat("Máximo de Ganancia del Ensemble:", format(ganancia_maxima, big.mark = ","), "en el envío:", indice_maximo, "\n")

lista_curvas_plot <- list()
lista_curvas_plot[[1]] <- cbind(tipo = "Ensemble", res_ensemble$data_plot)

for(s in names(todas_las_predicciones_semilla)) {
    res_semilla <- calc_curva_acum_filtrada(todas_las_predicciones_semilla[[s]], drealidad, umbral_ganancia = umbral_ganancia_base)
    lista_curvas_plot[[length(lista_curvas_plot) + 1]] <- cbind(tipo = paste0("Seed ", s), res_semilla$data_plot)
}

todas_las_curvas_plot <- rbindlist(lista_curvas_plot, use.names = TRUE, fill = TRUE)

datos_ensemble_full <- res_ensemble$data_full

In [ ]:
library(RColorBrewer) 

semillas_list <- unique(todas_las_curvas_plot$tipo)
semillas_list <- semillas_list[semillas_list != "Ensemble"] 
num_semillas <- length(semillas_list)

colores_semillas_paleta <- rep(brewer.pal(12, "Paired"), ceiling(num_semillas / 12))[1:num_semillas]

colores_final <- setNames(c("black", colores_semillas_paleta), c("Ensemble", semillas_list))
breaks_final <- c("Ensemble", semillas_list) 


p_acumulada_ensemble <- ggplot(
    data = todas_las_curvas_plot, 
    aes(x = indice, y = ganancia_acumulada, color = tipo, group = tipo)
) +
  
  geom_line(
    aes(linewidth = ifelse(tipo == "Ensemble", 1.5, 0.4), 
        alpha = ifelse(tipo == "Ensemble", 1.0, 0.7))
  ) +
  
  scale_color_manual(
    values = colores_final,
    breaks = breaks_final, 
    name = "Modelo"
  ) +
  
  scale_linewidth_identity() + 
  scale_alpha_identity() +
  
  geom_point(data = datos_ensemble_full[indice == indice_maximo], 
             aes(x = indice, y = ganancia_acumulada),
             color = "red", size = 4, inherit.aes = FALSE) + 
             
  annotate("label", 
           x = indice_maximo, 
           y = ganancia_maxima * 1.05,
           label = paste0("Máximo\n", format(ganancia_maxima, big.mark = ",", scientific = FALSE)),
           color = "red", 
           fontface = "bold",
           fill = "white",
           size = 4) +
           
  geom_segment(data = datos_ensemble_full[indice == indice_maximo], 
               aes(x = indice_maximo, y = ganancia_maxima,
                   xend = indice_maximo, yend = ganancia_maxima * 1.04),
               arrow = arrow(length = unit(0.3, "cm")),
               color = "red", 
               inherit.aes = FALSE) + 
  
  # Escalas y Etiquetas
  scale_x_continuous(labels = scales::comma, name = "Clientes ordenados por probabilidad (Índice / Envíos)") +
  scale_y_continuous(labels = scales::comma, name = "Ganancia Acumulada") +
  labs(
    title = paste0("Ganancia Acumulada (Semillas y Ensemble) con Filtrado - EXP ", PARAM$experimento)
  ) +
  theme_minimal() +
  theme(
    plot.title = element_text(face = "bold", size = 14),
    legend.position = "right",
    panel.grid.minor = element_line(color = "gray90") 
  )
print(p_acumulada_ensemble)

nombre_archivo_acum <- paste0("curva_ganancia_acumulada_ensemble_semillas__", PARAM$experimento, ".png")
ggsave(nombre_archivo_acum, p_acumulada_ensemble, width = 14, height = 8, dpi = 300)

cat("Gráfico de Ganancia Acumulada guardado en:", nombre_archivo_acum, "\n")

In [ ]:
Sys.time()